In [1]:
from pathlib import Path
import json
from PIL import Image
import numpy as np
import cv2
from random import randint

In [10]:
path_img = Path("/media/data/datasets/autoriaNumberplateOcrUa/train_val/")
path_new = Path("/media/data/datasets/autoriaNumberplateOcrUa/augmented/")

img_list = [file for file in path_img.iterdir()]# if file.suffix == ".png"]
augment_rate = 0.35
augment_qty = int(len(img_list) * augment_rate)
img_augmented = img_list[:augment_qty]
img_normal = img_list[augment_qty:]

for file in img_augmented:
    img = Image.open(file)
    margins = 500 # px
    img_bg = Image.new(mode="RGB", size=(img.width + margins * 2, img.height + margins * 2), color = 0)
    img_bg.paste(img, (margins, margins))
    img = np.array(img_bg)
    width = img.shape[1]
    height = img.shape[0]
    # shearY - 0.6; 0.4
    transformation_matrix = np.array([[(randint(400, 1000) / 1000), 0, 0], 
                                    [(randint(-600, 600) / 1000), 1, 0]], dtype=np.float32)
    img = cv2.warpAffine(img, transformation_matrix, (width, height))
    
    non_empty_columns = np.where(img.max(axis=0)>0)[0]
    non_empty_rows = np.where(img.max(axis=1)>0)[0]
    cropBox = (min(non_empty_rows), max(non_empty_rows), min(non_empty_columns), max(non_empty_columns))

    img = img[cropBox[0]:cropBox[1]+1, cropBox[2]:cropBox[3]+1 , :]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2.imwrite(path_new.joinpath(file.name), img)
    # break

for file in img_normal:
    img = cv2.imread(file)
    cv2.imwrite(path_new.joinpath(file.name), img)
    # break